# **Exercise 4. Fetching economical activity data from INE API**
This code retrieves the distribution of the population across different economic sectors and provinces in Spain, segmented by sex and type of employment. It uses the National Institute of Statistics (INE) API to fetch the data, processes it, and stores the result in a structured pandas DataFrame. The final DataFrame includes averaged percentages per year, which can be used to analyze trends over time.

# Relevance:
The data retrieved is useful for analyzing the distribution of the population by sector, sex, and employment type across different provinces in Spain. This can help in market segmentation, understanding regional economic activities, and tailoring marketing campaigns and product pricing strategies based on demographic and economic data.

In [27]:
import requests
import pandas as pd
import datetime

# URL to fetch the data
url = 'https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/65353'

# Send a GET request to the URL
reply = requests.get(url)

# Check if the request was successful
if reply.status_code == 200:
    data = reply.json()  # Parse the JSON response

    # List to store the processed data
    processed_data = []

    # Loop over each entry in the data and extract relevant values
    for item in data:
        nombre = item['Nombre']  # Get the name
        # Debug: print the 'Nombre' field to check its structure
        print(f"Nombre: {nombre}")

        for record in item['Data']:
            # Convert UNIX timestamp to date
            fecha = datetime.date.fromtimestamp(record['Fecha'] // 1000)  # Convert UNIX timestamp to date
            valor = record['Valor']  # Get the value
            year = record['Anyo']  # Get the year

            # Extract the components from 'Nombre'
            # Remove the prefix "Distribución porcentual de la población por sector económico."
            if nombre.startswith("Total Nacional."):
                nombre_clean = nombre.split("Distribución porcentual de la población por sector económico.")[1]
            else:
                nombre_clean = nombre.replace("Distribución porcentual de la población por sector económico.", "")

            # Split the cleaned 'Nombre' into components
            parts = nombre_clean.split(". ")

            if len(parts) >= 4:
                provincia = parts[0].strip()  # First part is the Provincia
                economical_sector = parts[2].strip()  # Last part is the sector
                sex = parts[1].strip()  # Second part is the sex
                type_of_employment = parts[4].strip()  # Third part is the type of employment

                # Append the data to the list
                processed_data.append({
                    'Provincia': provincia,
                    'Economical sector': economical_sector,
                    'Sex': sex,
                    'Type of employment': type_of_employment,
                    'Percent': valor,
                    'Year': year
                })

    # Convert the processed data to a pandas DataFrame
    df = pd.DataFrame(processed_data)

    # Group by 'Provincia', 'Economical sector', 'Sex', 'Type of employment', 'Year'
    # and calculate the mean of 'Percent' for each group
    df_grouped = df.groupby(['Provincia', 'Economical sector', 'Sex', 'Type of employment', 'Year'], as_index=False).agg({'Percent': 'mean'})


else:
    print(f"Failed to fetch data. Status code: {reply.status_code}")


Nombre: Total Nacional. Distribución porcentual de la población por sector económico. Ambos sexos. Agricultura. Porcentaje. Activos. 
Nombre: Total Nacional. Distribución porcentual de la población por sector económico. Ambos sexos. Industria. Porcentaje. Activos. 
Nombre: Total Nacional. Distribución porcentual de la población por sector económico. Ambos sexos. Construcción. Porcentaje. Activos. 
Nombre: Total Nacional. Distribución porcentual de la población por sector económico. Ambos sexos. Servicios. Porcentaje. Activos. 
Nombre: Total Nacional. Distribución porcentual de la población por sector económico. Ambos sexos. Parados que buscan primer empleo o han dejado su último empleo hace más de 1 año. Porcentaje. Activos. 
Nombre: Distribución porcentual de la población por sector económico. Albacete. Ambos sexos. Agricultura. Porcentaje. Activos. 
Nombre: Distribución porcentual de la población por sector económico. Albacete. Ambos sexos. Industria. Porcentaje. Activos. 
Nombre: Di

In [26]:
df_grouped

,Provincia,Economical sector,Sex,Type of employment,Year,Percent
0,Albacete,Agricultura,Ambos sexos,Activos,2008,7.050
1,Albacete,Agricultura,Ambos sexos,Activos,2009,5.775
2,Albacete,Agricultura,Ambos sexos,Activos,2010,6.925
3,Albacete,Agricultura,Ambos sexos,Activos,2011,7.175
4,Albacete,Agricultura,Ambos sexos,Activos,2012,8.100
...,...,...,...,...,...,...
4500,Ávila,Servicios,Ambos sexos,Activos,2020,62.850
4501,Ávila,Servicios,Ambos sexos,Activos,2021,62.825
4502,Ávila,Servicios,Ambos sexos,Activos,2022,67.625
4503,Ávila,Servicios,Ambos sexos,Activos,2023,68.425
